In [127]:
import happybase

connection = happybase.Connection('hbase-master',autoconnect=True)

In [95]:
#建立一個table
connection.create_table(
    'people',
    {
        'profile':dict()
    }
)

AlreadyExists: AlreadyExists(message=b'table name already in use')

In [128]:
table = connection.table('people')

In [12]:
with table.batch() as bat:
    bat.put('Candy', {'profile:gender': 'female', 'profile:age': '25', 'profile:addr': 'Taipei'})
    bat.put('Judy', {'profile:gender': 'female', 'profile:age': '34', 'profile:addr': 'Taipei'})
    bat.put('Tracy', {'profile:gender': 'female', 'profile:age': '18', 'profile:addr': 'Kaohsiung'})
    bat.put('Harrison', {'profile:gender': 'male', 'profile:age': '56', 'profile:addr': 'Tainan'})
    bat.put('Peter', {'profile:gender': 'male', 'profile:age': '54', 'profile:addr': 'Hualien'})
    bat.put('Berry', {'profile:gender': 'female', 'profile:age': '65', 'profile:addr': 'Taichuang'})

In [97]:
query_str = "SingleColumnValueFilter ('profile', 'addr', =, 'substring:Tai')"
query = table.scan(filter=query_str, limit=10)
for key, value in query:
    print(key, value)

b'Berry' {b'profile:addr': b'Taichuang', b'profile:age': b'65', b'profile:gender': b'female'}
b'Candy' {b'profile:addr': b'Taipei', b'profile:age': b'25', b'profile:gender': b'female'}
b'Harrison' {b'profile:addr': b'Tainan', b'profile:age': b'56', b'profile:gender': b'male'}
b'Judy' {b'profile:addr': b'Taipei', b'profile:age': b'34', b'profile:gender': b'female'}


In [99]:
query_str = "SingleColumnValueFilter('profile', 'age', >, 'binary:25')"
query = table.scan(filter=query_str, limit=10)
for key, value in query:
    print(key, value)

b'Berry' {b'profile:addr': b'Taichuang', b'profile:age': b'65', b'profile:gender': b'female'}
b'Harrison' {b'profile:addr': b'Tainan', b'profile:age': b'56', b'profile:gender': b'male'}
b'Judy' {b'profile:addr': b'Taipei', b'profile:age': b'34', b'profile:gender': b'female'}
b'Peter' {b'profile:addr': b'Hualien', b'profile:age': b'54', b'profile:gender': b'male'}


In [102]:
query = table.scan(filter="SingleColumnValueFilter('profile', 'addr', =, 'substring:Tai')" \
                  and "SingleColumnValueFilter('profile', 'age', =, 'binary:25')", limit=20)
for key, value in query:
    print(key, value)

b'Candy' {b'profile:addr': b'Taipei', b'profile:age': b'25', b'profile:gender': b'female'}


In [125]:
query = table.scan(columns=[b'profile:gender', b'profile:age'], \
           filter="SingleColumnValueFilter('profile', 'addr', =, 'substring:Tai')" \
              and "SingleColumnValueFilter('profile', 'age', >, 'binary:25')", limit=20)
#for key, value in query:
#    print(key, value)
    
#求和
sum = 0
for k, d in query:
    age = d[b'profile:age']
    sum = sum + int(age)
print("sum age:", sum)

sum age: 209


In [137]:
#計數
query = table.scan(columns=[b'profile:gender', b'profile:age'], filter="SingleColumnValueFilter('profile', 'addr', =, 'substring:Tai')") 
count = 0
for k, d in query:
    print(k, d[b'profile:gender'])
    if (str(d[b'profile:gender'], 'utf-8')) == 'female':
        count = count +1
print(count)

b'Berry' b'female'
b'Candy' b'female'
b'Harrison' b'male'
b'Judy' b'female'
b'Peter' b'male'
b'Tracy' b'female'
4


In [138]:
connection.close()